In [24]:
pwd

u'/Users/patrickkennedy/Desktop/Data_Science/Patently/code'

In [1]:
import os

import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
import pandas as pd
import numpy as np
import string
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics.pairwise import linear_kernel
import cPickle as pickle
import time
import msgpack
import gensim, logging
from nltk.corpus import stopwords

Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
import pandas as pd
import numpy as np
import string
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics.pairwise import linear_kernel
import cPickle as pickle
import time
import msgpack

def load_data(path=None):
    '''
    DOCSTRING: load_data

    Given the 'subset' or 'total' parameter, find the respective .csv file,
    read into a csv, parse out relevant fields and pickle the relevant objects.

    Returns: Dataframe object, abstracts array, descriptions array, claims array
    '''
    if path == 'subset':
        path = '../data/total_parsed_data_subset.csv'
    elif path == 'total':
        path = '../data/total_parsed_data.csv'
    else:
        print "ERROR: pass a valid path to data as a parameter"
        return

    df = pd.read_csv(path)
    df.fillna("", inplace=True)


    abstracts = df.abstract.values
    descriptions = df.description.values
    claims = df.claims.values
    
        

    return df, abstracts, descriptions, claims




def vectorize(text, tfidf=None):
    '''
    DOCSTRING: vectorize

    Given raw text, and the optional tfidf parameter, use TfidfVectorizer
    to vectorize text.  If tfidf parameter is present, use its vocab
    for the input text for relevant comparison.

    Returns: (fit_transformed text, tfidf object), (transformed text, __)
    '''
    if tfidf:
        return tfidf.transform(text)
    elif tfidf is None:
        tfidf = TfidfVectorizer(stop_words='english')
        return tfidf.fit_transform(text), tfidf



def get_similarity(vocab, idea, n_items=5):
    '''
    DOCSTRING: get_similarity

    For given vocab as tfidf sparse matrix and an input idea to test,
    check to make sure the sparse matrix column space is equal and
    use cosine similarity and n_items parameter to return the
    relevant cosine similarity scores and indices

    Returns: cosine similarity scores, indices
    '''
    if vocab.shape[1] == idea.shape[1]:
        pass
    else:
        print 'ERROR: shape mismatch'
        return

    cosine_similarity = vocab * idea.T
    cs_dense = np.array(cosine_similarity.todense())
    cs_array = np.array([float(i) for i in cs_dense])

    #indicate how many results to return... currently 10
    ind = np.argpartition(cs_array, -n_items)[-n_items:]

    #this prints out the top results (unsorted).. these are to be
    #transormed into scores
    sorted_ind = ind[np.argsort(cs_array[ind])][::-1]
    scores = cs_array[sorted_ind]
    indices = sorted_ind


    return scores, indices




def main(path, pkl):

    tic = time.clock()
    try:
        path = path
    except:
        print "ERROR: Specify data type [subset/total]"
        return

    try:
        pkl = pkl
    except:
        print "ERROR: Specify pickle behavior [True/False]"
        return


    if pkl == 'True':
        try:
            print 'Loading data...'
            df, abstracts, descriptions, claims = load_data(path)
            totals
            abstracts_tfidf, tfidf = vectorize(abstracts)

        except:
            print 'Error loading data!'
            return

        print 'Pickling data...'

        # think about writing a pickle function that loops
        # over a set of items passed in

        pickle.dump(abstracts_tfidf, open('../data/abstracts_tfidf.p', 'wb'))
        pickle.dump(tfidf, open('../data/tfidf.p', 'wb'))
        df.to_msgpack('../data/dataframe.p')
        print 'Finished pickling...'

    elif pkl == 'False':
        print 'Unpickling data...'
        abstracts_tfidf = pickle.load(open('../data/abstracts_tfidf.p', 'rb'))
        tfidf = pickle.load(open('../data/tfidf.p', 'rb'))
        df = pd.read_msgpack('../data/dataframe.p')
    else:
        print "Second argument to pickle must be [True/False]"
        return

    toc = time.clock()
    print 'User input (hardcoded)'
    text = ['Blood coagulation cold plasma device that kills bacteria']
    new_text_tfidf = vectorize(text, tfidf)

    print 'Getting similarity...'
    scores, indices = get_similarity(abstracts_tfidf, new_text_tfidf, 5)

    '''
    [Index([u'doc_number', u'date', u'publication_type', u'patent_length', u'title',
       u'abstract', u'description', u'claims']
    '''
    df_results = df.loc[indices][['doc_number', 'date', 'title', 'abstract']]
    df_results['score'] = scores
    print df_results
    print time.clock() - tic
    return df, abstracts_tfidf, tfidf

In [4]:
df, abstracts_tfidf, tfidf = main('subset', 'False')

Unpickling data...
User input (hardcoded)
Getting similarity...
      doc_number      date                                              title  \
35334   07482472  20090127               Substituted isoindoles and their use   
40916   08758603  20140624  Plasma purifying device and method of controll...   
73505   08697449  20140415       Optical blood coagulation monitor and method   
32283   08143347  20120327  Pegylation of recombinant blood coagulation fa...   
1095    07157456  20070102  Substituted oxazolidinones and their use in th...   

                                                abstract     score  
35334  The invention relates to coagulation of the bl...  0.384719  
40916  A plasma purifying device is provided that is ...  0.374798  
73505  An optical blood coagulation monitor and metho...  0.295731  
32283  The present invention relates to a proteinaceo...  0.293408  
1095   The invention relates to the field of blood co...  0.290961  
18.481031


In [5]:
text = 'Blood coagulation cold plasma device that kills bacteria'
new_text_tfidf = vectorize([text], tfidf)
scores, indices = get_similarity(abstracts_tfidf, new_text_tfidf, 5)
df_results = df.loc[indices][['doc_number', 'date', 'title', 'abstract']]
df_results['score'] = scores
print(df_results)


      doc_number      date                                              title  \
35334   07482472  20090127               Substituted isoindoles and their use   
40916   08758603  20140624  Plasma purifying device and method of controll...   
73505   08697449  20140415       Optical blood coagulation monitor and method   
32283   08143347  20120327  Pegylation of recombinant blood coagulation fa...   
1095    07157456  20070102  Substituted oxazolidinones and their use in th...   

                                                abstract     score  
35334  The invention relates to coagulation of the bl...  0.384719  
40916  A plasma purifying device is provided that is ...  0.374798  
73505  An optical blood coagulation monitor and metho...  0.295731  
32283  The present invention relates to a proteinaceo...  0.293408  
1095   The invention relates to the field of blood co...  0.290961  


In [6]:
toc = time.clock()
print("Putting words into lists...")
word_list = []
holder = []
for words in df.abstract.values:
    for word in words.lower().split():
        holder.append(word.strip(string.punctuation))
    word_list.append(holder)
    holder = []

print("Training the model...")
sw = stopwords.words('english')
bigram_transformer = gensim.models.Phrases(word_list)
model = gensim.models.Word2Vec(word_list, min_count=10,
                                size=200, workers=6)






Putting words into lists...
Training the model...


In [7]:
print("Process user text...")
text_list = text.lower().split()
text_ready = [w for w in text_list if w not in sw]
print("Process corpus...")
word_list_ready = [[w for w in words if w not in sw] for words in word_list]

scores = []

Process user text...
Process corpus...


In [8]:
word_list_ready

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['server-side',
  'recycle',
  'bin',
  'system',
  'retaining',
  'computer',
  'files',
  'information',
  'disclosed',
  'system',
  'comprises',
  'local',
  'computer',
  'system',
  'server',
  'including',
  'server-side',
  'recycle',
  'bin',
  'one',
  'persistent',
  'storage',
  'devices',
  'providing',
  'files',
  'directories',
  'protected',
  'present',
  'either',
  'part',
  'local',
  'computer',
  'system',
  'part',
  'server',
  'local',
  'computer',
  'system',
  'server',
  'may',
  'connected',
  'via',
  'wide',
  'area',
  'computer',
  'network',
  'local',
  'area',
  'network',
  'internet',
  'method',
  'combination',
  'methods',
  'file',
  'manager',
  'application',
  'running',
  'local',
  'computer',
  'system',
  'interacts',
  'file',
  'serving',
  'application',
  'server',
  'gener

In [ ]:
print("Getting scores...")
# for i in xrange(len(df.abstract.values)):
#     if i%10 == 0:
#         scores.append(model.wmdistance(text_ready, word_list_ready[i]))
        
        
scores = np.array([model.wmdistance(text_ready, word_list_ready[i]) for i in xrange(len(word_list_ready))])

Getting scores...


In [48]:
scores

array([ inf,  inf,  inf,  inf,  inf])

In [44]:
#indicate how many results to return... currently 10
scores_clean = []
for item in scores:
    if np.isnan(item):
        scores_clean.append(10000)
    else:
        scores_clean.append(item)
        
        
ind = np.argpartition(scores_clean, -5)[-5:]



In [47]:
scores

array([ inf,  inf,  inf,  inf,  inf])

In [43]:
#this prints out the top results (unsorted).. these are to be
#transormed into scores
# sorted_ind = ind[np.argsort(scores[ind])][::]
np.argsort(scores)[:5]

array([0, 1, 2, 3, 4])

In [28]:
scores_sorted = scores[sorted_ind]
indices = sorted_ind

df_results = df.loc[indices][['doc_number', 'date', 'title', 'abstract']]
df_results['score'] = scores
print(df_results)
print(time.clock() - toc)

       doc_number      date                  title abstract  score
98187    D0547819  20070731           Fishing lure             inf
119876   D0687443  20130806  Data accessing device             inf
119875   D0687429  20130806     Self-service kiosk             inf
119880   D0687499  20130806    Scalloped head stem             inf
119879   D0687485  20130806              Type font             inf
1242.473413
